In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1) # random as normal distribution but only in 2SD
    return tf.Variable(init_random_dist)
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

# conv func
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

# layer func
def convolutional_layer(input_x, shape):
    # [Height, Weight, Channel(l-1), Channel(l)]
    W = init_weights(shape)
    b = init_bias([shape[3]]) # shape[3] -> current # of channels
    return tf.nn.relu( conv2d(input_x, W) + b)
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

# Placeholders
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

# initiate layer
x_image = tf.reshape(x, [-1, 28, 28, 1])
convo_1 = convolutional_layer(x_image,shape=[6,6,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

convo_2 = convolutional_layer(convo_1_pooling, [6, 6, 32, 64])
convo_2_pooling = max_pool_2by2(convo_2)

# Why 7 by 7 image? Because we did 2 pooling layers, so (28/2)/2 = 7
# 64 then just comes from the output of the previous Convolution
convo_2_flat = tf.reshape(convo_2_pooling, [-1, 7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

# placeholder
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

y_pred = normal_full_layer(full_one_dropout, 10) # output 0-9

# Loss Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

init = tf.global_variables_initializer()

# Session
steps = 5000
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train, feed_dict={x:batch_x, y_true: batch_y, hold_prob: 0.5})
        
        # print output every 100 steps
        if i%100 == 0:
            print('Currently on step {}'.format(i))
            print('Accuracy is ')
            
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            print(sess.run(acc, feed_dict={x: mnist.test.images, y_true: mnist.test.labels, hold_prob: 1.0}))
            print('\n')

Currently on step 0
Accuracy is 
0.1334


Currently on step 100
Accuracy is 
0.8523


Currently on step 200
Accuracy is 
0.9096


Currently on step 300
Accuracy is 
0.9253


Currently on step 400
Accuracy is 
0.9322


Currently on step 500
Accuracy is 
0.9443


Currently on step 600
Accuracy is 
0.9505


Currently on step 700
Accuracy is 
0.9539


Currently on step 800
Accuracy is 
0.9581


Currently on step 900
Accuracy is 
0.9608


Currently on step 1000
Accuracy is 
0.9594


Currently on step 1100
Accuracy is 
0.9642


Currently on step 1200
Accuracy is 
0.9664


Currently on step 1300
Accuracy is 
0.9697


Currently on step 1400
Accuracy is 
0.9692


Currently on step 1500
Accuracy is 
0.9702


Currently on step 1600
Accuracy is 
0.9713


Currently on step 1700
Accuracy is 
0.9716


Currently on step 1800
Accuracy is 
0.973


Currently on step 1900
Accuracy is 
0.9751


Currently on step 2000
Accuracy is 
0.9738


Currently on step 2100
Accuracy is 
0.9734


Currently on step 2200
